In [127]:
# This is an implementation of TF-IDF based on the following link
# https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76
# If you failed to access the web page, you might access the pdf file in the following link
# https://drive.google.com/file/d/1J7J6p8hZ3lDmncxq1f6HHNstaGbtREHj/view?usp=sharing

import pandas as pd
import re

In [133]:
df = pd.read_csv('Dataset.csv') #read data
df #munculkan data

,No_IDLagu,Judul_Lagu,Bait_Reff,NIM,Nama
0,1,Kukira Kau Rumah,Kukira kau rumah\nNyatanya kau cuma aku sewa\n...,1301170262,Fadhli Kamil Maulana
1,2,Monolog,Alasan masih bersama\nBukan karena terlanjur l...,1301170262,Fadhli Kamil Maulana
2,3,Bila,"Bila saja dikau bisa berbicara, oh\nIzinkan da...",1301170262,Fadhli Kamil Maulana
3,4,Seberapa Pantas,Celakanya\nHanya kaulah yang benar-benar aku t...,1301174510,Naufal Adi Nugroho
4,5,Dekat di Hati,Aku di sini dan kau di sana\nHanya berjumpa vi...,1301174510,Naufal Adi Nugroho
...,...,...,...,...,...
70,71,Peri Cintaku,Aku untuk kamu\r\nKamu untuk aku\r\nNamun semu...,1301174687,Fauzi Cahya Miftahul Sidiq
71,72,Bahasa Kalbu,Percayalah\nHanya diriku paling mengerti\nKege...,1301174687,Fauzi Cahya Miftahul Sidiq
72,73,Indonesia Raya,Indonesia Raya\nMerdeka Merdeka\nTanahku negri...,1301171750,Muh. Agung Fazrulhaq
73,74,Kopi Dangdut,Api asmara yang dahulu pernah membara\r\nSemak...,1301171750,Muh. Agung Fazrulhaq


In [134]:
#drop feature yang tidak digunakan
data = df.drop(columns=['No_IDLagu','Judul_Lagu',	'NIM', 'Nama'])
data

,Bait_Reff
0,Kukira kau rumah\nNyatanya kau cuma aku sewa\n...
1,Alasan masih bersama\nBukan karena terlanjur l...
2,"Bila saja dikau bisa berbicara, oh\nIzinkan da..."
3,Celakanya\nHanya kaulah yang benar-benar aku t...
4,Aku di sini dan kau di sana\nHanya berjumpa vi...
...,...
70,Aku untuk kamu\r\nKamu untuk aku\r\nNamun semu...
71,Percayalah\nHanya diriku paling mengerti\nKege...
72,Indonesia Raya\nMerdeka Merdeka\nTanahku negri...
73,Api asmara yang dahulu pernah membara\r\nSemak...


In [135]:
#pre-processing data
import string
data['Bait_Reff'] = data['Bait_Reff'].apply(lambda x:x.lower())
data['Bait_Reff'].str.replace('[{}]'.format(string.punctuation), '')
data = data.replace('\n',' ', regex=True)
data = data.replace('\r','', regex=True)
data

,Bait_Reff
0,kukira kau rumah nyatanya kau cuma aku sewa da...
1,alasan masih bersama bukan karena terlanjur la...
2,"bila saja dikau bisa berbicara, oh izinkan dak..."
3,celakanya hanya kaulah yang benar-benar aku tu...
4,aku di sini dan kau di sana hanya berjumpa via...
...,...
70,aku untuk kamu kamu untuk aku namun semua apa ...
71,percayalah hanya diriku paling mengerti kegeli...
72,indonesia raya merdeka merdeka tanahku negriku...
73,api asmara yang dahulu pernah membara semakin ...


In [136]:
#TFIDF dengan library
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(strip_accents='unicode',stop_words='english')

In [137]:
tf_idf = vectorizer.fit_transform(data['Bait_Reff'])
dff = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())
dff

,abadi,ada,adakah,adalah,adanya,agar,agung,air,akan,akankah,akhir,akhirnya,aku,akunmu,akupun,alasan,alunan,andalkan,angan,antara,apa,apakah,apapun,api,arahmu,arti,asmara,atau,ataukah,awan,awas,ayo,bagai,bagi,bagiku,bahagia,baik,baikmu,baiknya,baja,...,terwujud,tetap,tiada,tiba,tidak,tinggal,tinggalkan,titik,tolong,tolonglah,tubuh,tubuhku,tuhan,tujuh,tuk,tuliskan,tulus,tumbuh,tunggu,tutur,ucap,ucapan,ujung,ulat,untuk,untukku,untukmu,untuknya,usai,wajahmu,wajar,waktu,walau,wanita,warna,ya,yakin,yakinlah,yang,yg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.055876,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.150673,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.098383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.052541,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.223666,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.085466,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.208099,0.0,0.0,0.0,0.084566,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.228036,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.150322,0.0,0.0,0.000000,0.000000,0.135118,0.0,0.108421,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.123304,0.0,0.0,0.0,0.0,0.0,0.176452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.188466,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159141,0.0,0.0,0.0,0.064670,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.060810,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.261459,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.171528,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.188579,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.306907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.163902,0.0
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.070996,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.349415,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.191445,0.066758,0.0
72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0

In [145]:
#Knn
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=6)
knn.fit(dff)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [146]:
#Cari Lagu dengan Id (input)
Cari_Lagu = 10

In [149]:
user_strain = data['Bait_Reff'][Cari_Lagu-1]

#lowercase dan hapus simbol data 
user_strain = user_strain.lower()
user_strain = [re.sub('[^a-zA-Z 0-9]', ' ', user_strain)]

pred_strains=knn.kneighbors((vectorizer.transform(user_strain)).todense())

List_ID = list(pred_strains[1][0])
List_ID.pop(0)
Recom_Id = tuple(List_ID)
print(Recom_Id) #output

(17, 70, 65, 39, 32)


**Tugas Besar 1: Implementasi Content Based Filtering dengan TF-IDF.**
1. Copy teks lirik "Reff" dari lagu berbahasa Indonesia (masing-masing 3 lagu) maksimum hari Rabu (4 nov 2020) jam9 pagi
2. Paste ke link berikut (Pastikan 1 lagu 1 cell lirik_ref, 1 cell = 1 document): 
https://docs.google.com/spreadsheets/d/14YBO5kKBLgaL3w__iLhwITRbnLOxMZPf778VKppZFaM/edit?usp=sharing
3. Cari informasi mengenai k-NN, dan yang diimplementasikan pada TF-IDF pada dataset di atas
4. Buat satu prosedur yang menerima ID dari lagu, dan kembalikan 5 ID (k=5) most-similar items (contoh: input ID lagu 1, output ID lagu 3,2,5,6,7 dimana ID-ID lagu ini similar to ID lagu 1)

**Dikumpulkan maksimum tanggal 16 November 2020 jam12.00 WIB di LMS** 

